### Ньяти Каелиле БВТ2201
### Курсовая Работа - СИАОД

In [1]:
import random
from collections import deque
import numpy as np

def generate_timetable_with_all_shifts():
    # Initialize drivers for night and day shifts (12-hour)
    night_drivers = deque(["12hr_Driver_1", "12hr_Driver_2", "12hr_Driver_3"])  # 3 Night-shift drivers
    day_drivers = deque(["12hr_Day_Driver_1", "12hr_Day_Driver_2", "12hr_Day_Driver_3"])  # 3 Day-shift drivers
    eight_hour_drivers = [
        "8hr_Driver_1", "8hr_Driver_2",  # 6:00-14:00
        "8hr_Driver_3", "8hr_Driver_4"   # 10:00-18:00
    ]
    
    schedule = []  # Store the timetable
    days = [
        "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
        "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
        "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
    ]
    
    # Rest pools for night shift and day shift drivers
    night_rest_pool = deque()
    day_rest_pool = deque()

    # Store the working drivers for each day (now includes shift times)
    day_drivers_schedule = {}

    # Assigning drivers for weekday (Monday-Friday) and weekend (Saturday-Sunday) 12-hour shifts
    for day in days:
        schedule.append(f"{day}:")
        day_drivers_schedule[day] = []  # Initialize the list for each day's working drivers

        # Reset lunch and break assignments at the start of each day
        lunch_drivers = []  # Reset lunch drivers for the day

        # Night shift assignment (if applicable)
        if night_rest_pool and len(night_rest_pool[0]["rest_days"]) == 2:
            rested_driver = night_rest_pool.popleft()["driver"]
            night_drivers.append(rested_driver)

        current_night_driver = night_drivers.popleft()
        schedule.append(f"  {current_night_driver} - night shift (18:00 to 6:00)")
        night_rest_pool.append({"driver": current_night_driver, "rest_days": []})

        # 8-hour day shift assignment (6:00-18:00 and 10:00-18:00)
        if "Saturday" not in day and "Sunday" not in day:  # Only Monday to Friday
            morning_shift_start_time = 6 * 60  # Start at 06:00 in minutes
            late_shift_start_time = (9 * 60) + 45  # Start at 10:00 in minutes

            for i in range(4):  # First four 8-hour drivers (6:00-18:00 and 10:00-18:00)
                if i < 2:
                    current_8hr_driver = eight_hour_drivers[i]
                    start_hour = morning_shift_start_time // 60
                    start_minute = morning_shift_start_time % 60
                    schedule.append(f"  {current_8hr_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02})")
                    day_drivers_schedule[day].append((current_8hr_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02}"))
                    morning_shift_start_time += 15  # Increment by 15 minutes for the next driver
                else:
                    current_8hr_driver = eight_hour_drivers[i]
                    start_hour = late_shift_start_time // 60
                    start_minute = late_shift_start_time % 60
                    schedule.append(f"  {current_8hr_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02})")
                    day_drivers_schedule[day].append((current_8hr_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 8) % 24:02}:{start_minute:02}"))
                    late_shift_start_time += 15  # Increment by 15 minutes for the next driver

        # 12-hour day shift assignment (for Monday-Friday)
        if "Saturday" not in day and "Sunday" not in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                # First 12-hour shift driver (starts at 06:00)
                current_day_shift_driver = day_drivers.popleft()
                start_hour = morning_shift_start_time // 60
                start_minute = morning_shift_start_time % 60
                schedule.append(f"  {current_day_shift_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02})")
                day_drivers_schedule[day].append((current_day_shift_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02}"))
                start_hour = (start_hour + 12) % 24  # Move the start hour 12 hours ahead for the next shift

                day_rest_pool.append({"driver": current_day_shift_driver, "rest_days": []})


        # Day shift assignment for weekends (Saturdays and Sundays)
        if "Saturday" in day or "Sunday" in day:
            if day_rest_pool and len(day_rest_pool[0]["rest_days"]) == 2:
                rested_day_driver = day_rest_pool.popleft()["driver"]
                day_drivers.append(rested_day_driver)

            if day_drivers:
                # First 12-hour shift driver (starts at 06:00)
                current_day_driver = day_drivers.popleft()
                start_hour = 6  # Start at 06:00
                start_minute = 0  # Start at 00 minutes
                schedule.append(f"  {current_day_driver} - day shift ({start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02})")
                day_drivers_schedule[day].append((current_day_driver, f"{start_hour:02}:{start_minute:02} to {(start_hour + 12) % 24:02}:{start_minute:02}"))
                start_hour = (start_hour + 12) % 24  # Move the start hour 12 hours ahead for the next shift

                day_rest_pool.append({"driver": current_day_driver, "rest_days": []})




        # Increment rest days for night-shift drivers
        for resting_driver in night_rest_pool:
            resting_driver["rest_days"].append(1)

        # Increment rest days for day-shift drivers
        for resting_driver in day_rest_pool:
            resting_driver["rest_days"].append(1)

        # Assign staggered lunch breaks
        lunch_drivers = assign_lunch_breaks(schedule, day_drivers_schedule[day])

        # Assign staggered 15-minute breaks for drivers without lunch
        assign_staggered_15_minute_breaks(schedule, lunch_drivers, day_drivers_schedule[day])

          # Assign staggered 15-minute breaks for night-shift drivers
        assign_staggered_15_minute_breaks_night(
            schedule,
            [(current_night_driver, "18:00", "6:00")]
        )

    return schedule


def assign_lunch_breaks(schedule, working_drivers):
    """
    Assign lunch breaks based on the day's working drivers.
    """
    
    lunch_drivers_today = []
            
    twelve_hour_day_driver = [
    (driver, start_time) for driver, start_time in working_drivers if "12hr" in driver
]
    if twelve_hour_day_driver:
        for driver, start_time in twelve_hour_day_driver:
            # Extract the actual start time (before " to")
            actual_start_time = start_time.split(" to ")[0]
            
            if actual_start_time == "06:00":
                # Assign lunch break from 13:00 to 14:00 for drivers starting at 06:00
                schedule.append(f"  {driver} - lunch break (13:00 to 14:00)")
                lunch_drivers_today.append(driver)
            else:
                # Assign lunch break from 13:30 to 14:30 for other start times
                schedule.append(f"  {driver} - lunch break (13:30 to 14:30)")
                lunch_drivers_today.append(driver)


    """
    # Assign lunch for one of the 10:00–18:00 shift drivers (14:00 to 15:00)
    afternoon_drivers = [
        driver for driver, start_time in working_drivers if int(start_time.split(":")[0]) == 10
    ]
    
    if afternoon_drivers:
        # Assign lunch to the first afternoon shift driver from 14:00 to 15:00
        schedule.append(f"  {afternoon_drivers[0]} - lunch break (14:00 to 15:00)")
        lunch_drivers_today.append(afternoon_drivers[0])
    """
    return lunch_drivers_today


def assign_staggered_15_minute_breaks(schedule, lunch_drivers, working_drivers):
    """
    Assign staggered 15-minute breaks to drivers not assigned lunch breaks.
    """
    # Initialize a list to store break assignments
    break_assignments = []
    
    # We will use a dictionary to track assigned break times to avoid conflicts
    assigned_breaks = {}

    # Filter out drivers who have lunch breaks
    eligible_drivers = [
        driver for driver, _ in working_drivers if driver not in lunch_drivers
    ]
    
    for driver, start_time in working_drivers:
        if driver not in eligible_drivers:
            continue

        # Extract the shift start time (assumes "HH:MM to HH:MM" format)
        if " to " in start_time:
            start_time = start_time.split(" to ")[0]  # Keep only the start time

        try:
            # Parse shift start time
            driver_start_hour = int(start_time.split(":")[0])
            driver_start_minute = int(start_time.split(":")[1])
        except ValueError:
            print(f"Skipping invalid start time for driver {driver}: {start_time}")
            continue

      # Dynamically calculate the end hour based on the start time and 8-hour shift
        driver_end_hour = driver_start_hour + 8
        driver_end_minute = driver_start_minute  # Assuming no additional minutes (full 8 hours)

        # Adjust the end time if the hours exceed 24 (wrap around)
        if driver_end_hour >= 24:
            driver_end_hour -= 24  # Wrap around to next day

        # List to store the assigned breaks for this driver
        breaks = []

        # Assign the first break (1hr 45mins after the start of the shift)
        current_hour = driver_start_hour
        current_minute = driver_start_minute + 105
        if current_minute >= 60:
            current_hour += current_minute // 60
            current_minute %= 60

        # Add the first break
        breaks.append((current_hour, current_minute))

        # Now assign up to 4 more breaks, spaced 1hr 45mins after the previous break's end time
        while len(breaks) < 6:
            last_break_hour, last_break_minute = breaks[-1]
            current_hour = last_break_hour
            current_minute = last_break_minute + 15 + 105  # Add 15 mins (break duration) + 1hr 45mins
            if current_minute >= 60:
                current_hour += current_minute // 60
                current_minute %= 60

            # Ensure the break fits within the shift's end time
            if current_hour >= driver_end_hour and (current_hour > driver_end_hour or current_minute > driver_end_minute):
                break

            # Add the next break
            breaks.append((current_hour, current_minute))
                
        # Assign breaks to the schedule while ensuring no overlap
        for break_hour, break_minute in breaks:
            formatted_start = f"{break_hour % 24:02}:{break_minute:02}"
            formatted_end_minute = break_minute + 15
            formatted_end_hour = break_hour
            if formatted_end_minute >= 60:
                formatted_end_hour += formatted_end_minute // 60
                formatted_end_minute %= 60
            formatted_end = f"{formatted_end_hour % 24:02}:{formatted_end_minute:02}"

            # Ensure the break time doesn't overlap with another driver
            while formatted_start in assigned_breaks:
                # If there's a conflict, move the break 15 minutes later
                break_minute += 15
                if break_minute >= 60:
                    break_hour += 1
                    break_minute -= 60
                formatted_start = f"{break_hour % 24:02}:{break_minute:02}"
                formatted_end_minute = break_minute + 15
                formatted_end_hour = break_hour
                if formatted_end_minute >= 60:
                    formatted_end_hour += formatted_end_minute // 60
                    formatted_end_minute %= 60
                formatted_end = f"{formatted_end_hour % 24:02}:{formatted_end_minute:02}"

            # Add the break to the schedule and track the assigned time
            break_assignments.append(f"  {driver} - 15-minute break ({formatted_start} to {formatted_end})")
            assigned_breaks[formatted_start] = driver
    
    # Append the break assignments to the schedule
    schedule.extend(break_assignments)


def assign_staggered_15_minute_breaks_night(schedule, night_drivers):
    """
    Assign staggered 15-minute breaks to night-shift drivers.
    Ensure no two drivers are on break at the same time.
    """
    break_assignments = []  # Store break assignments
    assigned_breaks = {}  # Track assigned break times to avoid conflicts

    for driver, shift_start, shift_end in night_drivers:
        # Parse the shift start and end times
        start_hour = int(shift_start.split(":")[0])
        start_minute = int(shift_start.split(":")[1])
        end_hour = int(shift_end.split(":")[0]) + 24  # Convert to 24-hour range for overnight shifts
        end_minute = int(shift_end.split(":")[1])

        # Initialize the current time for breaks (1hr 45mins after shift start)
        current_hour = start_hour
        current_minute = start_minute + 105  # Add 1hr 45mins
        if current_minute >= 60:
            current_hour += current_minute // 60
            current_minute %= 60

        # List to store this driver's assigned breaks
        breaks = []

        # Assign breaks at 1.5-hour intervals until the end of the shift
        while current_hour + current_minute / 60 < end_hour + end_minute / 60:
            # Format the current break start and end times
            break_start = f"{current_hour % 24:02}:{current_minute:02}"
            break_end_minute = current_minute + 15  # Add 15 minutes for the break duration
            break_end_hour = current_hour
            if break_end_minute >= 60:
                break_end_hour += break_end_minute // 60
                break_end_minute %= 60
            break_end = f"{break_end_hour % 24:02}:{break_end_minute:02}"

            # Ensure no overlap with other drivers
            while break_start in assigned_breaks:
                # Move the break 15 minutes later if there's a conflict
                current_minute += 15
                if current_minute >= 60:
                    current_hour += current_minute // 60
                    current_minute %= 60
                break_start = f"{current_hour % 24:02}:{current_minute:02}"
                break_end_minute = current_minute + 15
                break_end_hour = current_hour
                if break_end_minute >= 60:
                    break_end_hour += break_end_minute // 60
                    break_end_minute %= 60
                break_end = f"{break_end_hour % 24:02}:{break_end_minute:02}"

            # Assign the break
            breaks.append(f"  {driver} - 15-minute break ({break_start} to {break_end})")
            assigned_breaks[break_start] = driver

            # Increment time by 1hr 45mins for the next break (from the end of the current break)
            current_minute += 15 + 105  # Add 15 minutes (current break duration) + 1hr 45mins
            if current_minute >= 60:
                current_hour += current_minute // 60
                current_minute %= 60

        # Add this driver's breaks to the overall assignments
        break_assignments.extend(breaks)

    # Append the break assignments to the schedule
    schedule.extend(break_assignments)




def calculate_route_duration(time_of_day):
    """
    Calculate the route duration based on the time of day.
    Rush hours add 5 minutes; normal variability is ±10 minutes.
    """
    base_duration = 90  # 1.5 hours in minutes
    variability = random.randint(-10, 10)
    rush_hour_extra = 5 if time_of_day in ["7:00", "8:00", "09:00", "17:00", "18:00", "19:00"] else 0
    return base_duration + variability + rush_hour_extra


# Generate the updated timetable
timetable = generate_timetable_with_all_shifts()

# Print the combined timetable in the requested format
for entry in timetable:
    print(entry)


Monday (Week 1):
  12hr_Driver_1 - night shift (18:00 to 6:00)
  8hr_Driver_1 - day shift (06:00 to 14:00)
  8hr_Driver_2 - day shift (06:15 to 14:15)
  8hr_Driver_3 - day shift (09:45 to 17:45)
  8hr_Driver_4 - day shift (10:00 to 18:00)
  12hr_Day_Driver_1 - day shift (06:30 to 18:30)
  12hr_Day_Driver_1 - lunch break (13:30 to 14:30)
  8hr_Driver_1 - 15-minute break (07:45 to 08:00)
  8hr_Driver_1 - 15-minute break (09:45 to 10:00)
  8hr_Driver_1 - 15-minute break (11:45 to 12:00)
  8hr_Driver_1 - 15-minute break (13:45 to 14:00)
  8hr_Driver_2 - 15-minute break (08:00 to 08:15)
  8hr_Driver_2 - 15-minute break (10:00 to 10:15)
  8hr_Driver_2 - 15-minute break (12:00 to 12:15)
  8hr_Driver_2 - 15-minute break (14:00 to 14:15)
  8hr_Driver_3 - 15-minute break (11:30 to 11:45)
  8hr_Driver_3 - 15-minute break (13:30 to 13:45)
  8hr_Driver_3 - 15-minute break (15:30 to 15:45)
  8hr_Driver_3 - 15-minute break (17:30 to 17:45)
  8hr_Driver_4 - 15-minute break (12:15 to 12:30)
  8hr_Drive

In [10]:
import pandas as pd

# Define table structures
days = [
    "Monday (Week 1)", "Tuesday (Week 1)", "Wednesday (Week 1)", "Thursday (Week 1)", "Friday (Week 1)",
    "Saturday (Week 1)", "Sunday (Week 1)", "Monday (Week 2)", "Tuesday (Week 2)", "Wednesday (Week 2)",
    "Thursday (Week 2)", "Friday (Week 2)", "Saturday (Week 2)", "Sunday (Week 2)"
]

# Prepare tables for output
drivers_schedule = {
    "Driver": [],
    **{day: [] for day in days}
}
lunch_breaks = {
    "Driver": [],
    **{day: [] for day in days}
}
fifteen_min_breaks = {
    "Driver": [],
    **{day: [] for day in days}
}

# Collect timetable data from the generated schedule
timetable_data = generate_timetable_with_all_shifts()

# Process timetable into structured data
current_driver = None

# Initialize variables to track the current day
current_day = None

for line in timetable_data:
    line = line.strip()  # Clean up leading/trailing spaces
    line_cleaned = line.rstrip(":")  # Remove trailing colons if present

    if not line:
        continue  # Skip empty lines

    # Detect a new day
    if line_cleaned in days:
        current_day = line_cleaned
        print(f"Current day set to: {current_day}")
        continue  # Move to the next line

    if current_day is None:
        print(f"Skipping line as no day is set: {line}")
        continue  # Skip lines if no day is set yet

    # Process shift details
    if "night shift" in line or "day shift" in line:
        shift_type = "Night Shift" if "night shift" in line else "Day Shift"
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in drivers_schedule
        if driver not in drivers_schedule["Driver"]:
            drivers_schedule["Driver"].append(driver)
            print(f"Added driver {driver} to drivers_schedule.")
            for day in days:
                if day not in drivers_schedule:
                    print(f"Initializing day {day} in drivers_schedule.")
                    drivers_schedule[day] = []
                drivers_schedule[day].append("")  # Initialize all days with empty strings

        # Update the current day for the driver
        driver_index = drivers_schedule["Driver"].index(driver)
        drivers_schedule[current_day][driver_index] = f"{shift_type}: {time_range}"
        print(f"Updated {current_day} for driver {driver}: {drivers_schedule[current_day][driver_index]}")

    # Process lunch breaks
    elif "lunch break" in line:
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in lunch_breaks
        if driver not in lunch_breaks["Driver"]:
            lunch_breaks["Driver"].append(driver)
            print(f"Added driver {driver} to lunch_breaks.")
            for day in days:
                if day not in lunch_breaks:
                    print(f"Initializing day {day} in lunch_breaks.")
                    lunch_breaks[day] = []
                lunch_breaks[day].append("")  # Initialize all days with empty strings

        # Update the current day for the driver
        driver_index = lunch_breaks["Driver"].index(driver)
        lunch_breaks[current_day][driver_index] = time_range
        print(f"Updated {current_day} for driver {driver}: {lunch_breaks[current_day][driver_index]}")

    
    # Process 15-minute breaks
   # Process 15-minute breaks
    elif "15-minute break" in line:
        driver = line.split(" - ")[0].strip()
        time_range = line.split("(")[-1].strip(")")

        # Initialize driver if not already in fifteen_min_breaks["Driver"]
        if driver not in fifteen_min_breaks["Driver"]:
            fifteen_min_breaks["Driver"].append(driver)
            print(f"Added driver {driver} to fifteen_min_breaks.")
            
            # Ensure that each day for this driver is initialized as an empty string (not a list)
            for day in days:
                fifteen_min_breaks[day].append("")  # Ensure each driver gets an empty string for each day

        # Get the index for the driver in the "Driver" list
        driver_index = fifteen_min_breaks["Driver"].index(driver)

        # Ensure we append the break time to the correct day
        # Instead of appending a new time, join with the existing time
        if fifteen_min_breaks[current_day][driver_index]:
            fifteen_min_breaks[current_day][driver_index] += f", {time_range}"
        else:
            fifteen_min_breaks[current_day][driver_index] = time_range
        print(f"Updated {current_day} for driver {driver}: {fifteen_min_breaks[current_day][driver_index]}")



# Convert dictionaries to DataFrames
drivers_df = pd.DataFrame(drivers_schedule)
lunch_breaks_df = pd.DataFrame(lunch_breaks)
fifteen_min_breaks_df = pd.DataFrame(fifteen_min_breaks)

columns = ["Driver"] + days

# Restructure data to create the desired layout for shifts (Day & Night)
day_shift_rows = [["Day Shift"] + [""] * (len(columns) - 1)]    # Header for Day Shift
night_shift_rows = [["Night Shift"] + [""] * (len(columns) - 1)]  # Header for Night Shift

for _, row in drivers_df.iterrows():
    # Extract just the time for day shifts
    day_shift_row = [row["Driver"]]
    for shift in row[1:]:
        if "Day Shift" in shift:
            # Extract the time part
            day_shift_row.append(shift.split(": ")[-1])  
        else:
            day_shift_row.append("")  # Keep other cells empty

    # Add the row if it contains at least one non-empty cell after "Driver"
    if any(day_shift_row[1:]):  
        day_shift_rows.append(day_shift_row)

    # Extract just the time for night shifts
    night_shift_row = [row["Driver"]]
    for shift in row[1:]:
        if "Night Shift" in shift:
            # Extract the time part
            night_shift_row.append(shift.split(": ")[-1])  
        else:
            night_shift_row.append("")  # Keep other cells empty

    # Add the row if it contains at least one non-empty cell after "Driver"
    if any(night_shift_row[1:]):  
        night_shift_rows.append(night_shift_row)

# Combine into the final timetable structure for drivers
final_rows = day_shift_rows + night_shift_rows

# Create the final DataFrame for shifts
final_df = pd.DataFrame(final_rows, columns=columns)

# Filter and restructure the breaks data to include only relevant information
# For lunch breaks, include only non-empty values, skipping the "Driver" column
lunch_breaks_filtered = lunch_breaks_df[lunch_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]

# For 15-minute breaks, include only non-empty values, skipping the "Driver" column
fifteen_min_breaks_filtered = fifteen_min_breaks_df[fifteen_min_breaks_df.drop(columns=["Driver"]).notna().any(axis=1)]

# Convert filtered DataFrames to lists of rows for easy concatenation
lunch_breaks_rows = lunch_breaks_filtered.values.tolist()
fifteen_min_breaks_rows = fifteen_min_breaks_filtered.values.tolist()

# Combine headers and their respective rows
lunch_rows = [["Lunch Break"] + [""] * (len(columns) - 1)]  # Header for lunch breaks
min_break_rows = [["15 min breaks"] + [""] * (len(columns) - 1)]  # Header for 15 min breaks
final_breaks_rows = lunch_rows + lunch_breaks_rows + min_break_rows + fifteen_min_breaks_rows

# Create a DataFrame from the combined rows
breaks_final_df = pd.DataFrame(final_breaks_rows, columns=columns)


# Save to Excel
file_path = r"C:\Users\khaye\OneDrive\Desktop\Algorithms\5 semester\Driver_Schedule_Shifts.xlsx"
with pd.ExcelWriter(file_path) as writer:
    final_df.to_excel(writer, sheet_name="Driver Timetable", index=False)
    breaks_final_df.to_excel(writer, sheet_name="Breaks Schedule", index=False)

print("Drivers Schedule:", pd.DataFrame(drivers_schedule))
print("Lunch Breaks:", pd.DataFrame(lunch_breaks))
print("15-Min Breaks:", pd.DataFrame(fifteen_min_breaks))




Current day set to: Monday (Week 1)
Added driver 12hr_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Driver_1: Night Shift: 18:00 to 6:00
Added driver 8hr_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_1: Day Shift: 06:00 to 14:00
Added driver 8hr_Driver_2 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_2: Day Shift: 06:15 to 14:15
Added driver 8hr_Driver_3 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_3: Day Shift: 09:45 to 17:45
Added driver 8hr_Driver_4 to drivers_schedule.
Updated Monday (Week 1) for driver 8hr_Driver_4: Day Shift: 10:00 to 18:00
Added driver 12hr_Day_Driver_1 to drivers_schedule.
Updated Monday (Week 1) for driver 12hr_Day_Driver_1: Day Shift: 06:30 to 18:30
Added driver 12hr_Day_Driver_1 to lunch_breaks.
Updated Monday (Week 1) for driver 12hr_Day_Driver_1: 13:30 to 14:30
Added driver 8hr_Driver_1 to fifteen_min_breaks.
Updated Monday (Week 1) for driver 8hr_Driver_1: 07:4